In [6]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import transforms
import torch.nn.functional as F

# Vérifier si CUDA est disponible, sinon utiliser le CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# Architecture simplifiée du CNN
class MelanomaCNN(nn.Module):
    def __init__(self):
        super(MelanomaCNN, self).__init__()
        self.conv = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.fc = nn.Linear(16 * 112 * 112, 1)

    def forward(self, x):
        x = F.relu(self.conv(x))
        x = F.max_pool2d(x, 2)
        x = x.view(-1, 16 * 112 * 112)
        x = torch.sigmoid(self.fc(x))
        return x

class MelanomaTestDataset(Dataset):
    def __init__(self, img_dir, transform=None, limit=None):
        self.img_names = os.listdir(img_dir)[:limit]  # Ajoutez la limitation ici
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.img_names)

    def __getitem__(self, idx):
        img_name = os.path.join(self.img_dir, self.img_names[idx])
        image = Image.open(img_name)
        if self.transform:
            image = self.transform(image)
        return image, self.img_names[idx]

cuda


In [7]:
# Étape 1: Charger le modèle entraîné
model = MelanomaCNN()
model.load_state_dict(torch.load('melanoma_model.pth', map_location=device))  # Mettez à jour le chemin
model.to(device)  # Déplacer le modèle sur le GPU si disponible
model.eval()

MelanomaCNN(
  (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc): Linear(in_features=200704, out_features=1, bias=True)
)

In [9]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

test_dataset = MelanomaTestDataset(img_dir='../../data/test-resized/test-resized', transform=transform)  # Mettez à jour le chemin
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Étape 3: Effectuer des prédictions
predictions = []
image_names = []

with torch.no_grad():
    for data, names in test_loader:
        data = data.to(device)  # Déplacer les données sur le GPU si disponible
        outputs = model(data)
        # Convertir les sorties tensor en liste de valeurs flottantes
        predictions.extend(outputs.squeeze().tolist())
        # Retirer l'extension des noms d'image
        image_names.extend([n.split('.')[0] for n in names])

# Étape 4: Générer un fichier CSV pour la soumission
submission_df = pd.DataFrame({
    'image_name': image_names,
    'target': predictions
})

submission_df.to_csv('submission.csv', index=False)